## Basic modules

In [1]:
import numpy as np
import os
import scipy

# sklearn
from sklearn.svm import LinearSVC

# ignore convergence warnings from sklearn
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.ConvergenceWarning)

# pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

# pandas
import pandas as pd

# matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# seaborn
import seaborn as sns
sns.set()

# pickle
import pickle

# cv2
import cv2
from PIL import Image

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Reading data

The neural-activation (*in pickle format*) data consists of an organized dictionary with the following entries:

* `images_paths`: list containing paths to all the 1960 images
* `image_ctg`: numpy array containing class labels from 0 -> 6
* `image_splits` : 1960 x 10 numpy array containing 10 80:20 train:val splits used in the paper. Though I generate my own validation splits for computing the sit scores
* `features`: 168 dimensional(for multi-unit) neural_features for all the images i.e 1960 x 168 numpy array
* `categ_name_map`: dictionary mapping from numeric class label to class name e.g. face, animal etc.

The dataset consists of images belonging to 7 classes and 49 object types. The image paths are arranged in an order such that the images belonging to a particular object type are together. There are 40 images per object in the dataset, so images [1 - 40] belong to object 1, images [41 - 80] belong to object 2 and so on.

In [2]:
data_path = 'data/PLoSCB2014_data_20141216'
with open('data/PLoSCB2014_data_20141216/NeuralData_IT_multiunits.pkl','rb') as f:
    data = pickle.load(f)

In [3]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Preparing Input Images

For feeding the cadieu dataset images to the pretrained CNNs, we need to preprocess the images with appropriate reshaping, normalization and other data augmentation steps. In addition, we also need to convert the images to tensors, in order to use pytorch.

In [4]:
# define normalize transform to be used while feeding images to the pretrained CNN
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

# combine of transforms in a composition
transform = transforms.Compose([
            transforms.Resize(size=(224,224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])

# preprocessed input images list
X = []

for i,img_path in enumerate(data['image_paths']):
    img = transform(Image.open(os.path.join(data_path,img_path)))
    X.append(img)

# convert the list into a tensor
X = torch.stack(X)

print ("read {} images ... preprocessed input shape: {}".format(X.shape[0],X.shape))

read 1960 images ... preprocessed input shape: torch.Size([1960, 3, 224, 224])


## Load the pretrained model

There are 2 steps to be done here:

* Load the pretrained model e.g. alexnet,vgg16,resnet50 etc.
* Change it appropriately in order to extract appropriate features

In [6]:
# load model definitions
from utils.loader import ModelLoader

# create ModelLoader instance
model_loader = ModelLoader()

# define model
model = model_loader.load('')

# transfer model to device
model = model.to(device)

# set model to eval mode
model.eval()

AssertionError: 

## Extract features

In [ ]:
# total number of images
total_num_images = X.shape[0]

# batch size for extracting features
batch_size = 8

# number of batches
num_batches = total_num_images//batch_size

# model features
model_features = []

for i in range(num_batches):
    # sample batch and put to device
    x_batch = X[i*batch_size:(i+1)*batch_size].float().to(device)
    
    # extract features for the batch
    out = model(x_batch)
    
    # detach output and convert to numpy
    out = out.detach().cpu().numpy()
    
    # store model features for the current batch
    model_features.append(out)

# concatenate the model features for all batches
model_features = np.concatenate(model_features).squeeze()

print ("extracting features for {} images ... model features shape: {}"
                                   .format(model_features.shape[0],model_features.shape))

In [ ]:
np.any(model_features<0)

In [ ]:
with open('densenet201_feat.pkl','wb') as f:
    pickle.dump(model_features,f)

## Read neural features

In [ ]:
neural_features = data['features']
print ("read neural features for {} images with shape: {}".format(neural_features.shape[0],neural_features.shape))

## Compute Representational Dissimilarity Matrix (RDM)

In [ ]:
def get_rdm(features):
    # number of objects
    num_obj = 49
    
    # number of images per object
    num_imgs_per_obj = int(features.shape[0]/num_obj)
    
    # compute avg features per object
    avg_features = np.zeros((num_obj,features.shape[1]))
    for i in range(num_obj):
        avg_features[i] = np.mean(features[i*num_imgs_per_obj:(i+1)*num_imgs_per_obj],axis=0)
    
    # compute correlation matrix
    correlation_matrix = np.corrcoef(avg_features)
    
    # compute rdm matrix
    rdm = 1 - correlation_matrix
    
    return rdm

## Adding noise correction to model features

In [ ]:
def noise_correction(model_features,neural_features):
    # noise variance estimation parameters for multiunit
    a = 0.14
    b = 0.92
    
    # number of trials
    T = 47
    
    # estimated noise variance
    noise_var = np.mean((a*neural_features+b)**2)/T
    
    # total variance of signal + noise in the neural features
    sig_noise_var = np.var(neural_features)
    
    # expected signal variance for model representations
    expected_sig_var = sig_noise_var - noise_var
    
    # scaling model representations to match expected signal variance
    model_features = np.sqrt(expected_sig_var/np.var(model_features))*model_features
    
    # adding noise to model representations
    noise = np.random.randn(model_features.shape[0],model_features.shape[1])
    noise = (a*model_features+b)*noise/np.sqrt(T)
    model_features += noise
    
    return model_features

### Sampling different train/validation Splits

In [ ]:
def get_train_val_split(total_num_imgs=1960,val_ratio=0.2):
    # number of objects
    num_obj = 49
    
    # number of images per object
    num_imgs_per_obj = int(total_num_imgs/num_obj)
    
    # number of validation images for each object according to val_ratio
    num_val_imgs_per_obj = int(num_imgs_per_obj*val_ratio)
    
    # compute validation mask s.t. val_mask = 1 for validation images and o/w 0
    val_mask = np.zeros(total_num_imgs,dtype=int)
    
    for obj_count in range(num_obj):
        choose = np.random.choice(range(num_imgs_per_obj),num_val_imgs_per_obj,replace=False)
        choose += num_imgs_per_obj*obj_count
        val_mask[choose] = 1
    
    # compute train mask as inverse of val_mask
    train_mask = (val_mask==0).astype(int)
    
    return train_mask,val_mask

## Compute Similarity to IT Dissimilarity Matrix (SIT)

In [ ]:
# number of different validation splits
num_val_splits = 1000

# store sit scores for different validation splits
sit_scores = []

for i in range(num_val_splits):
    # get validation split
    _,val_mask = get_train_val_split(val_ratio=0.2)
    
    # get model and neural features for validation images
    val_model_features = model_features[val_mask==1]
    val_neural_features = neural_features[val_mask==1]
    
    # apply noise correction using validation model and neural features
    val_model_features = noise_correction(val_model_features,val_neural_features)
    
    # compute RDM matrices for neural and model representations
    rdm_neural = get_rdm(val_neural_features)
    rdm_model = get_rdm(val_model_features)
    
    # get upper triangular matrix values for model and neural rdm
    iu1 = np.triu_indices(49,k=1)
    rdm_neural_triu = rdm_neural[iu1]
    rdm_model_triu = rdm_model[iu1]
    
    # compute sit score and store the result
    sit_score = scipy.stats.spearmanr(rdm_model_triu,rdm_neural_triu).correlation
    sit_scores.append(sit_score)

# print the mean and standard deviation of sit scores
print ("sit_mean: {}\t sit_std: {}".format(np.mean(sit_scores),np.std(sit_scores)))

## Linear SVM accuracy score

In order to see how good any set of features is, we compute the Linear SVM accuracy obtained for the classification task on the cadieu dataset (7 classes).

In [ ]:
def linear_svm_score(features,labels,neural_features=None,num_subsampled_feat=168):
    
    # number of validation splits
    num_val_splits = 10
    
    # accuracy scores
    acc_scores = []
    
    # apply noise correction
    if neural_features is not None:
        features = noise_correction(features,neural_features)
        
    for _ in range(num_val_splits):
        # get train:test split with ratio 80:20
        train_mask,test_mask = get_train_val_split(val_ratio=0.2)

        # get training and test datasets
        X_train,y_train = features[train_mask==1],labels[train_mask==1]
        X_test,y_test = features[test_mask==1],labels[test_mask==1]
        
        # get train:val split with ratio 80:20
        train_mask,val_mask = get_train_val_split(X_train.shape[0],val_ratio=0.2)

        # get training and val datasets
        X_val,y_val = X_train[val_mask==1],y_train[val_mask==1]
        X_train,y_train = X_train[train_mask==1],y_train[train_mask==1]
        
        # number of feature subsamples
        num_feat_samples = 10
        
        for i in range(num_feat_samples):
            # get a subsample
            feat_subsample = np.random.choice(range(features.shape[1]),num_subsampled_feat,replace=False)
            
            # get subsampled train,validation and test datasets
            X_train_subsample = X_train[:,feat_subsample]
            X_val_subsample = X_val[:,feat_subsample]
            X_test_subsample = X_test[:,feat_subsample]
            
            # range to sample regularization parameter C
            C_range = [1e-3,1e-2,1e-1,1e0,1e1,1e2]
            
            # store val acc scores to choose the best C
            val_acc_scores = []
            
            for C in C_range:
                # linear SVM classifier
                clf = LinearSVC(C=C,max_iter=1000)

                # fit training data
                clf.fit(X_train_subsample,y_train)

                # get mean accuracy on validation data
                val_acc = clf.score(X_val_subsample,y_val)
                
                # store val_acc
                val_acc_scores.append(val_acc)
            
            # choose best C
            best_C = C_range[np.argmax(val_acc_scores)]
            
            # linear SVM classifier for best C
            clf = LinearSVC(C=best_C,max_iter=5000)

            # fit training data
            clf.fit(X_train_subsample,y_train)
            
            # get mean accuracy on test data
            test_acc = clf.score(X_test_subsample,y_test)
            
            # store accuracy
            acc_scores.append(test_acc)
    
    return np.mean(acc_scores),np.std(acc_scores)

In [ ]:
# get linear svm accuracy for model features 
acc_mean,acc_std = linear_svm_score(model_features,data['image_ctg'],neural_features)
print ("Model features: linear svm accuracy mean: {} \t std: {}".format(acc_mean,acc_std))

In [ ]:
# get linear svm accuracy for IT neural features 
acc_mean,acc_std = linear_svm_score(neural_features,data['image_ctg'])
print ("Neural features: linear svm accuracy mean: {} \t std: {}".format(acc_mean,acc_std))

In [ ]:
data_path = 'data/PLoSCB2014_data_20141216'
with open('data/PLoSCB2014_data_20141216/NeuralData_V4_multiunits.pkl','rb') as f:
    data_ = pickle.load(f)
    
v4_features = data_['features']

In [ ]:
# get linear svm accuracy for v4 neural features 
acc_mean,acc_std = linear_svm_score(v4_features,data['image_ctg'],num_subsampled_feat=128)
print ("Neural features: linear svm accuracy mean: {} \t std: {}".format(acc_mean,acc_std))